<a href="https://colab.research.google.com/github/ptrick/faceanim/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ffmpy &> /dev/null
!git init -q .
!git remote add origin https://github.com/ptrick/first-order-model
!git pull -q origin master

In [10]:
import IPython.display
import PIL.Image
import cv2
import imageio
import io
import numpy
import os.path
import requests
import skimage.transform
import warnings
from base64 import b64encode
from demo import load_checkpoints, make_animation
from ffmpy import FFmpeg
from google.colab import files, output
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")

img_path = "input/image.png"
vid_path = "input/input.mp4"


def select_image(filename):
  return resize(PIL.Image.open(filename).convert("RGB"))


def resize(image, size=(256, 256)):
  w, h = image.size
  d = min(w, h)
  r = ((w - d) // 2, (h - d) // 2, (w + d) // 2, (h + d) // 2)
  return image.resize(size, resample=PIL.Image.LANCZOS, box=r)


def generate():
  filename = 'vox-cpk.pth.tar'
  if not os.path.isfile(filename):
    download = requests.get(requests.get(
        'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://yadi.sk/d/lEw8uRm140L_eQ&path=/' + filename).json().get('href'))
    with open(filename, 'wb') as checkpoint:
      checkpoint.write(download.content)
  reader = imageio.get_reader(vid_path, mode='I', format='FFMPEG')
  fps = reader.get_meta_data()['fps']
  driving_video = []
  for frame in reader:
    driving_video.append(frame)
  generator, kp_detector = load_checkpoints(
      config_path='config/vox-256.yaml', checkpoint_path=filename)
  predictions = make_animation(
    skimage.transform.resize(numpy.asarray(select_image(img_path)), (256, 256)),
    [skimage.transform.resize(frame, (256, 256)) for frame in driving_video],
    generator,
    kp_detector,
    relative=relative.value,
    adapt_movement_scale=adapt_movement_scale.value)
  imageio.mimsave('temp.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
  FFmpeg(inputs={'temp.mp4': None, selected_video: None}, outputs={'output.mp4': '-c copy -y'}).run()


In [ ]:
generate()